In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

df = pd.read_csv('RELIANCE.csv')
df.columns

Index(['SYMBOL', ' SERIES', ' DATE1', ' PREV_CLOSE', ' OPEN_PRICE',
       ' HIGH_PRICE', ' LOW_PRICE', ' LAST_PRICE', ' CLOSE_PRICE',
       ' AVG_PRICE', ' TTL_TRD_QNTY', ' TURNOVER_LACS', ' NO_OF_TRADES',
       ' DELIV_QTY', ' DELIV_PER'],
      dtype='object')

In [2]:
df.drop(['SYMBOL', ' SERIES', ' DATE1'], inplace=True, axis=1)

In [3]:
df_Y = df[' CLOSE_PRICE'].copy()

In [4]:
df_split_Y = scaler.fit_transform(df_Y.to_numpy().reshape(-1,1))
df_split_Y = pd.DataFrame(df_split_Y)

In [5]:
train_Y = df_split_Y.iloc[:487]
test_Y = df_split_Y.iloc[487:609]

In [6]:
from sklearn.model_selection import train_test_split

X = []
Y = []
window_size=100
for i in range(1 , len(df) - window_size -1 , 1):
    first = df.iloc[i,2]
    temp = []
    temp2 = []
    for j in range(window_size):
        temp.append((df.iloc[i + j, 2] - first) / first)
    temp2.append((df.iloc[i + window_size, 2] - first) / first)
    X.append(np.array(temp).reshape(100, 1))
    Y.append(np.array(temp2).reshape(1, 1))
    
# Y = df_train[' CLOSE_PRICE'].copy()
# Y = df_train[' CLOSE_PRICE'][:609].copy().values
# Y = Y.reshape((609,1,1))

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

train_X = np.array(x_train)
test_X = np.array(x_test)
# train_Y = np.array(y_train)
# test_Y = np.array(y_test)

train_X = train_X.reshape(train_X.shape[0],1,100,1)
test_X = test_X.reshape(test_X.shape[0],1,100,1)

print(len(train_X))
print(len(test_X))

487
122


In [7]:
dfx = pd.DataFrame(train_X.reshape((487,100)))
dfy = pd.DataFrame(test_X.reshape((122,100)))

In [8]:
df_split_X = pd.DataFrame(scaler.fit_transform(df))

In [9]:
mx = pd.DataFrame(np.array(df_split_X.iloc[:487]))
train_X = pd.concat([dfx, mx], axis=1)

my = pd.DataFrame(np.array(df_split_X.iloc[487:609]))
test_X = pd.concat([dfy,my], axis=1)

In [10]:
train_X.shape

(487, 112)

In [11]:
train_Y.shape

(487, 1)

In [12]:
test_X.shape

(122, 112)

In [13]:
test_Y.shape

(122, 1)

In [14]:
train_X = np.array(train_X)
test_X = np.array(test_X)
train_Y = np.array(train_Y)
test_Y = np.array(test_Y)

train_X = train_X.reshape(train_X.shape[0],1,112,1)
test_X = test_X.reshape(test_X.shape[0],1,112,1)

In [15]:
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, Bidirectional, TimeDistributed
from tensorflow.keras.layers import MaxPooling1D, Flatten
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.metrics import RootMeanSquaredError

model = tf.keras.Sequential()

# Creating the Neural Network model here...
# CNN layers
model.add(TimeDistributed(Conv1D(64, kernel_size=3, activation='relu', input_shape=(None, 112, 1))))
model.add(TimeDistributed(MaxPooling1D(2)))
model.add(TimeDistributed(Conv1D(128, kernel_size=3, activation='relu')))
model.add(TimeDistributed(MaxPooling1D(2)))
model.add(TimeDistributed(Conv1D(64, kernel_size=3, activation='relu')))
model.add(TimeDistributed(MaxPooling1D(2)))
model.add(TimeDistributed(Flatten()))
# model.add(Dense(5, kernel_regularizer=L2(0.01)))

# LSTM layers
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(100, return_sequences=False)))
model.add(Dropout(0.5))

#Final layers
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mse', metrics=['mse', 'mae'])

history = model.fit(train_X, train_Y, validation_data=(test_X,test_Y), epochs=100,batch_size=32, verbose=1, shuffle =True)

Metal device set to: Apple M1 Pro
Epoch 1/100


2022-10-25 22:25:02.296915: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-10-25 22:25:02.297047: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2022-10-25 22:25:02.393617: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-10-25 22:25:04.237964: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-25 22:25:04.596362: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-25 22:25:04.607756: I tensorflow/core/grappler/o

16/16 [==============================] - ETA: 0s - loss: 0.1522 - mse: 0.1522 - mae: 0.3163

2022-10-25 22:25:06.079083: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-25 22:25:06.191682: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-25 22:25:06.199837: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-25 22:25:06.245901: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-10-25 22:25:06.254277: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


16/16 [==============================] - 4s 88ms/step - loss: 0.1522 - mse: 0.1522 - mae: 0.3163 - val_loss: 0.0502 - val_mse: 0.0502 - val_mae: 0.1833
Epoch 2/100
16/16 [==============================] - 0s 31ms/step - loss: 0.0515 - mse: 0.0515 - mae: 0.1872 - val_loss: 0.0466 - val_mse: 0.0466 - val_mae: 0.1689
Epoch 3/100
16/16 [==============================] - 0s 30ms/step - loss: 0.0381 - mse: 0.0381 - mae: 0.1570 - val_loss: 0.0239 - val_mse: 0.0239 - val_mae: 0.1235
Epoch 4/100
16/16 [==============================] - 0s 29ms/step - loss: 0.0158 - mse: 0.0158 - mae: 0.0982 - val_loss: 0.0071 - val_mse: 0.0071 - val_mae: 0.0598
Epoch 5/100
16/16 [==============================] - 0s 30ms/step - loss: 0.0050 - mse: 0.0050 - mae: 0.0492 - val_loss: 0.0053 - val_mse: 0.0053 - val_mae: 0.0644
Epoch 6/100
16/16 [==============================] - 0s 29ms/step - loss: 0.0035 - mse: 0.0035 - mae: 0.0420 - val_loss: 0.0035 - val_mse: 0.0035 - val_mae: 0.0488
Epoch 7/100
16/16 [=========

Epoch 49/100
16/16 [==============================] - 0s 29ms/step - loss: 1.7552e-04 - mse: 1.7552e-04 - mae: 0.0105 - val_loss: 0.0015 - val_mse: 0.0015 - val_mae: 0.0350
Epoch 50/100
16/16 [==============================] - 0s 29ms/step - loss: 1.0397e-04 - mse: 1.0397e-04 - mae: 0.0077 - val_loss: 0.0013 - val_mse: 0.0013 - val_mae: 0.0314
Epoch 51/100
16/16 [==============================] - 0s 29ms/step - loss: 9.9098e-05 - mse: 9.9098e-05 - mae: 0.0077 - val_loss: 0.0013 - val_mse: 0.0013 - val_mae: 0.0327
Epoch 52/100
16/16 [==============================] - 0s 29ms/step - loss: 1.1968e-04 - mse: 1.1968e-04 - mae: 0.0084 - val_loss: 0.0018 - val_mse: 0.0018 - val_mae: 0.0382
Epoch 53/100
16/16 [==============================] - 0s 28ms/step - loss: 1.6111e-04 - mse: 1.6111e-04 - mae: 0.0099 - val_loss: 0.0026 - val_mse: 0.0026 - val_mae: 0.0470
Epoch 54/100
16/16 [==============================] - 0s 29ms/step - loss: 2.1293e-04 - mse: 2.1293e-04 - mae: 0.0115 - val_loss: 0.001

16/16 [==============================] - 0s 29ms/step - loss: 8.6772e-05 - mse: 8.6772e-05 - mae: 0.0073 - val_loss: 0.0017 - val_mse: 0.0017 - val_mae: 0.0378
Epoch 97/100
16/16 [==============================] - 0s 29ms/step - loss: 9.4732e-05 - mse: 9.4732e-05 - mae: 0.0072 - val_loss: 0.0020 - val_mse: 0.0020 - val_mae: 0.0408
Epoch 98/100
16/16 [==============================] - 0s 29ms/step - loss: 1.3047e-04 - mse: 1.3047e-04 - mae: 0.0080 - val_loss: 0.0018 - val_mse: 0.0018 - val_mae: 0.0386
Epoch 99/100
16/16 [==============================] - 0s 29ms/step - loss: 1.0122e-04 - mse: 1.0122e-04 - mae: 0.0078 - val_loss: 0.0020 - val_mse: 0.0020 - val_mae: 0.0404
Epoch 100/100
16/16 [==============================] - 0s 31ms/step - loss: 1.2799e-04 - mse: 1.2799e-04 - mae: 0.0084 - val_loss: 6.7564e-04 - val_mse: 6.7564e-04 - val_mae: 0.0205
